# Ripple phase modulation

In [1]:
import pandas as pd
import sys
import pickle
sys.path.append("D:/ryanh/github/ripple_analyses")
from functions import *
from matplotlib import pyplot as plt
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
import glob as glob
from scipy.signal import hilbert
from scipy.ndimage import gaussian_filter1d
import itertools
from scipy import stats

from neurodsp.filt import filter_signal
from neurodsp.timefrequency import phase_by_time

from pycircstat import descriptive as desc
from pycircstat import distributions as dist
from pycircstat import tests as circ_test


Bad key "text.kerning_factor" on line 4 in
D:\ryanh\Anaconda3\envs\ripple_analysis\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


## Load data

In [2]:
data_path = 'F:/Projects/PAE_PlaceCell/ProcessedData/'
save_path = "F:/Projects/PAE_PlaceCell/analysis/ripple_modulation_phase/"
df_cell_class = pd.read_csv(r'F:\Projects\PAE_PlaceCell\analysis\cell_recruitment\processed\pyr_int_df.csv')
display(df_cell_class)


df = pd.read_csv('F:/Projects/PAE_PlaceCell/analysis/swr_data/post_processed/swr_df.csv')
display(df)


,Unnamed: 0,session,tetrode,cell,acg_tau_decay,acg_tau_rise,acg_c,acg_d,acg_asymptote,acg_refrac,...,trough,spkW,short_isi,lv,cv,avg_fr,asymmetry,n_spikes,cell_type,area
0,0,LEM3116_S20180715121821,TT1.mat,1,31.546309,0.573929,7.484840,1.810424,4.267133,3.757487,...,-2.393640,0.777777,0.021529,1.149097,1.535871,2.529014,3.181064,7340,pyr,ca1
1,1,LEM3116_S20180715121821,TT1.mat,2,2.066006,2.058184,99.344603,1.086215,2.731016,7.293023,...,-2.027969,0.930761,0.010545,1.041924,1.479007,0.392445,1.947498,1139,pyr,ca1
2,2,LEM3116_S20180715121821,TT2.mat,1,26.506166,0.781068,15.805209,1.504109,4.955974,4.657924,...,-2.491995,0.777777,0.041928,1.235779,1.773778,2.580697,3.908416,7490,pyr,ca1
3,3,LEM3116_S20180715121821,TT3.mat,1,4.712647,3.339123,44.065945,3.111166,0.573573,1.285490,...,-2.784299,0.571703,0.000555,1.032228,0.999013,0.621228,3.730098,1803,pyr,ca1
4,4,LEM3116_S20180715121821,TT3.mat,2,69.711440,1.119072,2.272114,4.029701,6.440918,4.588725,...,-2.683800,0.684153,0.011062,0.983793,1.406165,2.959360,3.743105,8589,pyr,ca1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8508,8508,RH16_S20161207130000,TT7.mat,5,13.648150,19.260993,4.054789,2.763695,3.311529,0.086722,...,-2.886261,0.529358,0.000000,1.059228,5.973929,0.789008,4.629547,1252,pyr,ca1
8509,8509,RH16_S20161207130000,TT7.mat,6,465.286016,6.540656,9.476762,1.811303,12.260592,0.000135,...,-3.028445,0.271706,0.004489,0.735687,23.134516,13.056442,2.818769,20718,int,ca1
8510,8510,RH16_S20161207130000,TT8.mat,1,40.117754,49.985020,7.880123,1.435894,3.685590,0.000087,...,-2.461559,0.286010,0.000000,0.955257,6.281825,0.626417,21.188431,994,int,ca1
8511,8511,RH16_S20161207130000,TT8.mat,2,5.123286,5.122541,5.461216,5.218173,9.395189,0.011953,...,-2.361157,0.701930,0.021382,0.999326,31.814075,1.533272,2.350860,2433,pyr,ca1


,Unnamed: 0,ripple_number,start_time,end_time,ripple_duration,max_emg,peak_time,peak_channel,peak_amplitude,peak_freq,clipped,session,area,ep_type,group,rat
0,0,22,211.247200,211.327200,0.080000,0.990871,211.296000,3,5.566472,213.605249,0.000000,LEM3116_S20180715121821,ca1,track,control,LEM3116
1,1,60,517.919200,518.311200,0.392000,0.995188,518.087200,7,29.168512,203.155556,0.000000,LEM3116_S20180715121821,ca1,track,control,LEM3116
2,2,72,607.938400,608.326400,0.388000,0.990928,607.964000,3,7.932019,103.616382,0.000000,LEM3116_S20180715121821,ca1,track,control,LEM3116
3,3,80,675.985600,676.139200,0.153600,0.994272,676.074400,7,5.077734,173.913383,0.000000,LEM3116_S20180715121821,ca1,track,control,LEM3116
4,4,100,933.226400,933.424000,0.197600,0.992893,933.278400,7,27.285162,119.721685,0.000000,LEM3116_S20180715121821,ca1,pedestal_2,control,LEM3116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18199,18728,25,255.450400,255.599200,0.148800,0.314032,255.533600,7,8.094016,105.746020,0.000000,RH16_S20161126145021,ca1,track,pae,RH16
18200,18729,22,149.384000,149.505600,0.121600,0.133732,149.436800,6,12.060743,186.017137,0.000000,RH16_S20161126150318,ca1,track,pae,RH16
18201,18730,32,233.419200,233.576000,0.156800,0.260312,233.496000,7,8.145433,110.793201,0.000000,RH16_S20161126150318,ca1,track,pae,RH16
18202,18731,36,270.288000,270.410400,0.122400,0.065024,270.328800,0,4.390508,174.380797,0.000000,RH16_S20161126150318,ca1,track,pae,RH16


In [3]:
def get_good_channels(shank):
    #extract values from dictionary
    an_array = np.array(list(shank.values()),dtype=object)
    
    #loop through array to pull out individual channel        
    good_ch = []
    for i in range(len(an_array)):
        for x in range(len(an_array[i])):
            good_ch.append(an_array[i][x])
        
    return good_ch

def smoothAngularTuningCurves(tuning_curve,deviation=3):
    n_bins = len(tuning_curve)
    padded = np.hstack([tuning_curve,tuning_curve,tuning_curve])
    padded = gaussian_filter1d(padded, deviation)
    return padded[n_bins:n_bins+n_bins]

def circ_interp(x1,theta,x2):
    '''
    Input:
        x1: linear x variable
        theta: angles in radians; same len as x1
        x2: new linear x variable 
    Output:
        new theta (-pi to pi)
    '''
    # polar to cartesian, then interp, then cartesian to polar
    new_theta = np.arctan2(np.interp(x2,x1,np.sin(theta)), np.interp(x2,x1,np.cos(theta)))
    # wrap -pi to pi and return
    return (new_theta + np.pi) % (2 * np.pi) - np.pi

    
def spike_phase_maps(spikes,phase,ts,df,session,fs=1250,n_bins=60):

    bin_edges = np.linspace(-np.pi,np.pi,n_bins+1)
    bin_width = np.diff(bin_edges)[0]
    bin_centers = np.linspace(-np.pi+bin_width/2 ,np.pi-bin_width/2, n_bins)
    
    # loop through spikes
    spk_phase_all = []
    phase_counts = []
    binned_phase = []
    smoothed_phase = []
    peak_phase = []
    spk_count = []
    resultant = []
    kappa = []
    rayleigh_z = []
    rayleigh_pval = []
    
    for spk in spikes:
        spk_phase = []
        occ_phase = []
        for ripple in df[df.session == session].itertuples():
            # restrict spikes to current ripple
            current_spikes = spk[(spk >= ripple.peak_time - 0.1) & (spk <= ripple.peak_time + 0.1)]
            # collect interp phase for each spike
            spk_phase.append(circ_interp(ts,phase[:,int(ripple.peak_channel)],current_spikes))
            # collect occ phases for normalization
            occ_phase.append(phase[(ts >= ripple.peak_time - 0.1) & (ts <= ripple.peak_time + 0.1),int(ripple.peak_channel)])

        # flatten phases into nice list    
        x = list(itertools.chain(*spk_phase)) 
        x2 = list(itertools.chain(*occ_phase))    
        # bin from -pi to pi
        y, _ = np.histogram(x, bins=bin_edges)
        y2, _ = np.histogram(x2, bins=bin_edges)
        
        # convert to firing rate in Hz
        tuning = y / (y2 / fs)
        
        # smooth and find max phase
        smooth_y = smoothAngularTuningCurves(tuning)
        max_phase = bin_centers[np.argmax(smooth_y)]
        
        pval, z = circ_test.rayleigh(bin_centers, w=smooth_y, d=bin_width)

        n_spikes = y.sum()
        
        # append for each cell
        spk_phase_all.append(x)
        phase_counts.append(y)
        binned_phase.append(tuning)
        smoothed_phase.append(smooth_y)
        peak_phase.append(max_phase)
        spk_count.append(n_spikes)

        # circ stats
        resultant.append(desc.resultant_vector_length(bin_centers, w=smooth_y, d=bin_width,axis=0))
        kappa.append(dist.kappa(bin_centers, w=smooth_y))
        rayleigh_z.append(z)
        rayleigh_pval.append(pval)

    return spk_phase_all,binned_phase,phase_counts,smoothed_phase,peak_phase,spk_count,resultant,kappa,rayleigh_z,rayleigh_pval

In [ ]:
# loop though sessions

def main(session,df,data_path,savepath):
    base = os.path.basename(session)
    os.path.splitext(base)
    save_file = save_path + os.path.splitext(base)[0] + '.pkl'

    # check if saved file exists
    if os.path.exists(save_file):
        return

    path = get_session_path(data_path+session+'.mat')

    # load xml which has channel & fs info
    channels,fs,shank = loadXML(path)

    # get good channels
    good_ch = get_good_channels(shank)

    lfp,ts = loadLFP(glob.glob(path +'\*.lfp')[0], n_channels=channels,
                     channel=good_ch, frequency=fs)
    
    phase = np.stack([phase_by_time(lfp_,fs,f_range=(80,250),remove_edges=False) for lfp_ in lfp.T])
    phase = phase.T
    
    # load spikes
    spikes = get_spikes(data_path+session+'.mat')
    
    spk_phase_all,binned_phase,phase_counts,smoothed_phase,peak_phase,spk_count,resultant,kappa,rayleigh_z,rayleigh_pval = spike_phase_maps(spikes,phase,ts,df,session,fs=fs)
        
    # save all cells in a session to file    
    with open(save_file, 'wb') as f:
        pickle.dump(spk_phase_all, f)
        pickle.dump(binned_phase, f)  
        pickle.dump(phase_counts,f)
        pickle.dump(smoothed_phase, f)  
        pickle.dump(peak_phase, f)
        pickle.dump(spk_count, f)
        pickle.dump(resultant, f)
        pickle.dump(kappa, f)  
        pickle.dump(rayleigh_z, f)   
        pickle.dump(rayleigh_pval, f)

sessions = pd.unique(df.session)
# for session in sessions:
#     main(session,df,data_path,save_path)
    
num_cores = multiprocessing.cpu_count()         
processed_list = Parallel(n_jobs=num_cores)(delayed(main)(session,df,data_path,save_path) for session in sessions)


In [ ]:
def load_in_everything(save_path):
    
    spk_phase_all = []
    binned_phase = []
    phase_counts = []
    smoothed_phase = []
    peak_phase = []
    spk_count = []
    resultant = []
    kappa = []
    rayleigh_z = []
    rayleigh_pval = []
    
    sessions = glob.glob(save_path + '*.pkl')
    for session in sessions:
        with open(session, 'rb') as f:
            spk_phase_all.append(pickle.load(f))
            binned_phase.append(pickle.load(f))
            phase_counts.append(pickle.load(f))
            smoothed_phase.append(pickle.load(f))
            peak_phase.append(pickle.load(f))
            spk_count.append(pickle.load(f))
            resultant.append(pickle.load(f))
            kappa.append(pickle.load(f))
            rayleigh_z.append(pickle.load(f))
            rayleigh_pval.append(pickle.load(f))
    
    kappa_new = []
    for i in range(len(kappa)):
        for j in range(len(kappa[i])):
            kappa_new.append(kappa[i][j][0])
        
    binned_phase = np.vstack(binned_phase)    
    phase_counts = np.vstack(phase_counts)    
    smoothed_phase = np.vstack(smoothed_phase)
    peak_phase = np.hstack(peak_phase)
    spk_count = np.hstack(spk_count)
    resultant = np.hstack(resultant)
#     kappa = np.hstack(kappa)
#     kappa = list(itertools.chain(*kappa))
    rayleigh_z = np.hstack(rayleigh_z)
    rayleigh_pval = np.hstack(rayleigh_pval)
    
    
    mod_dict = {
        "spk_phase": spk_phase_all,
        "binned_phase": binned_phase,
        "phase_counts": phase_counts,
        "smoothed_phase": smoothed_phase,
        "peak_phase": peak_phase,
        "spk_count": spk_count,
        "resultant": resultant,
        "kappa": kappa_new,
        "rayleigh_z": rayleigh_z,
        "rayleigh_pval": rayleigh_pval
    }
    return mod_dict

mod_dict = load_in_everything(save_path)

mod_dict.keys()

In [ ]:
for item in mod_dict.keys():
    print(item,len(mod_dict[item]))

### remove spk_phase for now

In [ ]:
del mod_dict['spk_phase'] 

In [ ]:
# metrics from analysis
df_cell_class['peak_phase'] = mod_dict['peak_phase']
df_cell_class['spk_count'] = mod_dict['spk_count']
df_cell_class['resultant'] = mod_dict['resultant']
df_cell_class['kappa'] = mod_dict['kappa']
df_cell_class['rayleigh_z'] = mod_dict['rayleigh_z']
df_cell_class['rayleigh_pval'] = mod_dict['rayleigh_pval']

# group
control=('RH13','RH14','LS21','LS23','LE2821','LE2823','LEM3116','LEM3120','LEM3216')
pae=('RH11','RH16','LS17','LS19','LE2813','LEM3124','LEM3206','LEM3246')
df_cell_class['group'] = np.ones_like(df_cell_class.session)
regstr = '|'.join(control)
idx = df_cell_class['session'].str.upper().str.contains(regstr)    
df_cell_class.loc[idx,'group'] = 'control'
regstr = '|'.join(pae)
idx = df_cell_class['session'].str.upper().str.contains(regstr)    
df_cell_class.loc[idx,'group'] = 'pae'

# number of ripples 
n_ripples = [sum(df.session == session) for session in df_cell_class.session]
df_cell_class['n_ripples'] = n_ripples

In [ ]:
idx = (df_cell_class['area'] == 'ca1') | (df_cell_class['area'] == 'ca3')

for item in mod_dict.keys():
    print(item)
    mod_dict[item] = np.array(mod_dict[item])[idx]

df_cell_class = df_cell_class[idx]

In [ ]:
from scipy import stats
mod_dict['smoothed_phase_zscore'] = stats.zscore(mod_dict['smoothed_phase'],axis=1)
mod_dict['binned_phase_zscore'] = stats.zscore(mod_dict['binned_phase'],axis=1)

In [ ]:
import matplotlib as mpl

class MidpointNormalize(mpl.colors.Normalize):
    '''
    https://stackoverflow.com/questions/7404116/defining-the-midpoint-of-a-colormap-in-matplotlib
    '''
    def __init__(self, vmin, vmax, midpoint=0, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        normalized_min = max(0, 1 / 2 * (1 - abs((self.midpoint - self.vmin) / (self.midpoint - self.vmax))))
        normalized_max = min(1, 1 / 2 * (1 + abs((self.vmax - self.midpoint) / (self.midpoint - self.vmin))))
        normalized_mid = 0.5
        x, y = [self.vmin, self.midpoint, self.vmax], [normalized_min, normalized_mid, normalized_max]
        return np.ma.masked_array(np.interp(value, x, y))

In [ ]:
n_ripples = 50
n_spikes = 100

good_cells = ((df_cell_class.n_ripples >= n_ripples) &
              (df_cell_class.n_spikes >= n_spikes) &
             (df_cell_class.rayleigh_pval < 0.05))

def plot_mod_matrix(df_cell_class,mod_dict,good_cells,group,area,cell_type,ax,x_off=False):
    norm = MidpointNormalize(vmin=-2.5, vmax=2, midpoint=0)

    h0_current = mod_dict['smoothed_phase_zscore'][good_cells &
                           (df_cell_class.cell_type == cell_type) &
                           (df_cell_class.area == area) &
                           (df_cell_class.group == group)]
    # get and sort by mean of center 3 columns
    center_idx = int(h0_current.shape[1]/2)
    idx = np.mean(h0_current[:,center_idx-1:center_idx+2],axis=1)
    idx = np.argsort(idx)
    # get x axis
    extent = [-np.pi , np.pi, 0 , len(idx)]
    # plot matrix
    ax.imshow(h0_current[idx,:],aspect='auto',extent=extent,cmap=plt.get_cmap('PuOr'),norm=norm)
    # restrict to +-100ms
#     ax.set_xlim(-0.1,0.1)
    ax.set_yticks([h0_current.shape[0]])
    if x_off:
        ax.axes.get_xaxis().set_ticks([])
    
def plot_mod_mean_std(df_cell_class,mod_dict,good_cells,group,area,cell_type,ax,color):
    h0_current = mod_dict['smoothed_phase_zscore'][good_cells &
                       (df_cell_class.cell_type == cell_type) &
                       (df_cell_class.area == area) &
                       (df_cell_class.group == group)]
    
    n_bins = 60
    bin_edges = np.linspace(-np.pi,np.pi,n_bins+1)
    bin_width = np.diff(bin_edges)[0]
    bin_centers = np.linspace(-np.pi+bin_width/2 ,np.pi-bin_width/2, n_bins)
    
#     bin_centers = np.linspace(-.5,.5,mod_dict['binned_phase_zscore'].shape[1]) 
    m = np.nanmean(h0_current,axis=0)
    v = np.nanstd(h0_current,axis=0) / np.sqrt(h0_current.shape[0])
    ax.plot(bin_centers,m, color = color, linewidth = 1)
    ax.fill_between(bin_centers, m-v, m+v, color = color, alpha = 0.5, linewidth = 0)
    ax.axvline(0, linestyle = '--', linewidth = 1, alpha = 0.5, color = 'grey')
    ax.set_title(group+' '+area)
    ax.set_xlim(-np.pi,np.pi)
    ax.set_ylim(-2.5,2)
    ax.axes.get_xaxis().set_ticks([])
    

fig, axs = plt.subplots(3,4, figsize=set_size('thesis', fraction=3, subplots=(3,4)), edgecolor='k')
fig.subplots_adjust(hspace = .05, wspace=.5)
ax = axs.ravel()  

cmap = plt.get_cmap("cool_r",2)

plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'control','ca1','int',ax[0],cmap(0))
ax[0].set_ylabel('z-score')

plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'control','ca1','pyr',ax[0],cmap(1))

plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'pae','ca1','int',ax[1],cmap(0))
plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'pae','ca1','pyr',ax[1],cmap(1))

plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'control','ca3','int',ax[2],cmap(0))
plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'control','ca3','pyr',ax[2],cmap(1))

plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'pae','ca3','int',ax[3],cmap(0))
plot_mod_mean_std(df_cell_class,mod_dict,good_cells,'pae','ca3','pyr',ax[3],cmap(1))


plot_mod_matrix(df_cell_class,mod_dict,good_cells,'control','ca1','int',ax[4],True)
plot_mod_matrix(df_cell_class,mod_dict,good_cells,'control','ca1','pyr',ax[8])
ax[8].set_xlabel('Phase (deg)')
ax[8].set_ylabel('neurons')

plot_mod_matrix(df_cell_class,mod_dict,good_cells,'pae','ca1','int',ax[5],True)
plot_mod_matrix(df_cell_class,mod_dict,good_cells,'pae','ca1','pyr',ax[9],True)

plot_mod_matrix(df_cell_class,mod_dict,good_cells,'control','ca3','int',ax[6],True)
plot_mod_matrix(df_cell_class,mod_dict,good_cells,'control','ca3','pyr',ax[10],True)

plot_mod_matrix(df_cell_class,mod_dict,good_cells,'pae','ca3','int',ax[7],True)
plot_mod_matrix(df_cell_class,mod_dict,good_cells,'pae','ca3','pyr',ax[11],True)

import matplotlib as mpl

norm = MidpointNormalize(vmin=-2.5, vmax=2, midpoint=0)

# norm = mpl.colors.Normalize(vmin=-0.5, vmax=4.5)
sm = plt.cm.ScalarMappable(cmap=plt.get_cmap('PuOr'), norm=norm)
sm.set_array([])
plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
cax = plt.axes([0.83, 0.1, 0.02, 0.52])
cbar = plt.colorbar(sm,cax=cax)
cbar.set_label('z-score', rotation=270, labelpad=20)
# cbar.outline.set_visible(False)

# plt.savefig(r'D:\ryanh\GoogleDrive\school_work\UNM\Lab\Projects\Dissertation\dissertation\inkscape_figs\ripple_recuitment\fr_mod.svg',dpi=300,bbox_inches='tight')


# testing below

In [ ]:
session = 'LEM3206_S20190720132154'
path = get_session_path(data_path+session+'.mat')

# load xml which has channel & fs info
channels,fs,shank = loadXML(path)

# get good channels
good_ch = get_good_channels(shank)

lfp,ts = loadLFP(glob.glob(path +'\*.lfp')[0], n_channels=channels,
                 channel=good_ch, frequency=fs)

phase = np.stack([phase_by_time(lfp_,fs,f_range=(80,250),remove_edges=False) for lfp_ in lfp.T])
phase = phase.T

In [ ]:
spikes = get_spikes(data_path+session+'.mat')

In [ ]:
n_spikes = [s.shape for s in spikes]

np.argmax(n_spikes)

In [ ]:
spk_phase = []
occ_phase = []

spk = spikes[14]

for ripple in df[df.session == session].itertuples():
    # restrict spikes to current ripple
    current_spikes = spk[(spk >= ripple.peak_time - 0.1) & (spk <= ripple.peak_time + 0.1)]
    # collect interp phase for each spike
    spk_phase.append(np.interp(current_spikes,ts,phase[:,int(ripple.peak_channel)]))

    # collect occ phases for normalization
    occ_phase.append(phase[(ts >= ripple.peak_time - 0.1) & (ts <= ripple.peak_time + 0.1),int(ripple.peak_channel)])

x1 = list(itertools.chain(*spk_phase)) 



spk_phase = []
occ_phase = []

for ripple in df[df.session == session].itertuples():
    # restrict spikes to current ripple
    current_spikes = spk[(spk >= ripple.peak_time - 0.1) & (spk <= ripple.peak_time + 0.1)]
    # collect interp phase for each spike
#     spk_phase.append(np.interp(current_spikes,ts,phase[:,int(ripple.peak_channel)]))
    # pull out phases
    theta = phase[:,int(ripple.peak_channel)]
    # get sin and cos then interp spike times
    spk_phase_ = np.arctan2(np.interp(current_spikes,ts,np.sin(theta)), np.interp(current_spikes,ts,np.cos(theta)))
    # wrap -pi to pi
    spk_phase_ = (spk_phase_ + np.pi) % (2 * np.pi) - np.pi
    spk_phase.append(spk_phase_)

    # collect occ phases for normalization
    occ_phase.append(phase[(ts >= ripple.peak_time - 0.1) & (ts <= ripple.peak_time + 0.1),int(ripple.peak_channel)])

x2 = list(itertools.chain(*spk_phase)) 


from scipy import interpolate
spk_phase = []
occ_phase = []
for ripple in df[df.session == session].itertuples():
    # restrict spikes to current ripple
    current_spikes = spk[(spk >= ripple.peak_time - 0.1) & (spk <= ripple.peak_time + 0.1)]
    # collect interp phase for each spike
#     spk_phase.append(np.interp(current_spikes,ts,phase[:,int(ripple.peak_channel)]))
    
    f = interpolate.interp1d(ts, phase[:,int(ripple.peak_channel)],kind='nearest')
    spk_phase.append(f(current_spikes))
    
    # collect occ phases for normalization
    occ_phase.append(phase[(ts >= ripple.peak_time - 0.1) & (ts <= ripple.peak_time + 0.1),int(ripple.peak_channel)])

x3 = list(itertools.chain(*spk_phase)) 


In [ ]:
plt.plot(x1,'.k')
plt.plot(x2,'.r')

plt.figure()
plt.scatter(x1,x2)

plt.figure()
plt.scatter(x2,x3)

In [ ]:
n_bins = 60
bin_edges = np.linspace(-np.pi,np.pi,n_bins+1)
bin_width = np.diff(bin_edges)[0]
bin_centers = np.linspace(-np.pi+bin_width/2 ,np.pi-bin_width/2, n_bins)
y, _ = np.histogram(x1, bins=bin_edges)

plt.plot(y)

y, _ = np.histogram(x2, bins=bin_edges)
plt.plot(y)

y, _ = np.histogram(x3, bins=bin_edges)
plt.plot(y)

In [ ]:
# theta = (position_df.hd * (np.pi / 180))
theta = phase[:,int(ripple.peak_channel)]
plt.figure()
plt.plot(theta[0:220])
plt.figure()
phases = np.arctan2(np.sin(theta), np.cos(theta))
# phases = (2*np.pi + phases) * (phases < 0) + phases*(phases > 0)

phases = (phases + np.pi) % (2 * np.pi) - np.pi
plt.plot(phases[0:220])